In [13]:
import os
import shutil
import cv2
import numpy as np
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [14]:
# define the base path to the input dataset and then use it to derive
# the path to the input images and annotation CSV files
BASE_PATH = "VOC2012"
IMAGES_PATH = os.path.sep.join([BASE_PATH, "JPEGimages"])
IMAGES_PATH_REDUCED = os.path.sep.join([BASE_PATH, "Images_Reduced"])
ANNOTS_PATH = os.path.sep.join([BASE_PATH, "Annotations"])
ANNOTS_PATH_REDUCED = os.path.sep.join([BASE_PATH, "Annotations_Reduced"])

# define the path to the base output directory
BASE_OUTPUT = "output"
os.makedirs(BASE_OUTPUT, exist_ok=True)
# define the path to the output model, label encoder, plots output
# directory, and testing image paths
MODEL_PATH = os.path.sep.join([BASE_OUTPUT, "detector.pth"])
LE_PATH = os.path.sep.join([BASE_OUTPUT, "le.pickle"])
PLOTS_PATH = os.path.sep.join([BASE_OUTPUT, "plots"])
os.makedirs(PLOTS_PATH, exist_ok=True)
TEST_PATHS = os.path.sep.join([BASE_OUTPUT, "test_paths.txt"])

Below we are going to preprocess the data in a way that we have the two datasets we need. First, the whole dataset with images with several objects and parts and other with the images with only one object and one part in it. This is done because the Pytorch architecture only works with one object per image, but YOLO handles multiple objects.

In [15]:
def has_single_bounding_box(object_element):
    # Check if an object has only one bounding box (ignoring parts)
    return len(object_element.findall('part')) == 0

def filter_annotations(input_folder_annotations, output_folder_annotations, input_folder_images, output_folder_images, limit_number = 10000):
    os.makedirs(output_folder_annotations, exist_ok=True)
    os.makedirs(output_folder_images, exist_ok=True)
    count = 0
    for filename in os.listdir(input_folder_annotations):
        if count >= limit_number:
            continue
        if filename.endswith(".xml"):
            xml_path = os.path.join(input_folder_annotations, filename)
            tree = ET.parse(xml_path)
            root = tree.getroot()

            # Check if the annotation has only one object with a single bounding box
            objects = root.findall('.//object')
            if len(objects) == 1 and has_single_bounding_box(objects[0]):
                output_path = os.path.join(output_folder_annotations, filename)
                # Copy the XML file to the output folder
                os.makedirs(output_folder_annotations, exist_ok=True)
                with open(output_path, 'wb') as output_file:
                    output_file.write(ET.tostring(root))
                   # Copy the corresponding JPEG image to the "JPEGImages" folder
                image_filename = os.path.splitext(filename)[0] + ".jpg"
                image_path_src = os.path.join(input_folder_images, image_filename)
                image_path_dest = os.path.join(output_folder_images, image_filename)
                shutil.copy(image_path_src, image_path_dest)
                count += 1 
input_folder_a = ANNOTS_PATH
output_folder_a = ANNOTS_PATH_REDUCED

input_folder_i = IMAGES_PATH
output_folder_i = IMAGES_PATH_REDUCED

filter_annotations(input_folder_a, output_folder_a, input_folder_i, output_folder_i, limit_number = 4000)

IMAGES_PATH_REDUCED_TEST = os.path.sep.join([BASE_PATH, "Images_Reduced_Test"])
ANNOTS_PATH_REDUCED_TEST = os.path.sep.join([BASE_PATH, "Annotations_Reduced_Test"])

input_folder_a = ANNOTS_PATH
output_folder_a = ANNOTS_PATH_REDUCED_TEST

input_folder_i = IMAGES_PATH
output_folder_i = IMAGES_PATH_REDUCED_TEST

filter_annotations(input_folder_a, output_folder_a, input_folder_i, output_folder_i, limit_number = 200)

First, we do it for the reduced dataset to compare with the other architecture.

In [16]:
# Initialize lists to store data
data = []
labels = []
bboxes = []
imagePaths = []

# loop over all XML files in the annotations directory
for xmlPath in paths.list_files(ANNOTS_PATH_REDUCED, validExts=(".xml")):
    # load the contents of the current XML annotations file
    tree = ET.parse(xmlPath)
    root = tree.getroot()

    # extract information from XML
    filename = root.find("filename").text
    label = root.find(".//name").text
    xmin = float(root.find(".//xmin").text)
    ymin = float(root.find(".//ymin").text)
    xmax = float(root.find(".//xmax").text)
    ymax = float(root.find(".//ymax").text)

    # derive the path to the input image, load the image (in OpenCV format), and grab its dimensions
    imagePath = os.path.sep.join([IMAGES_PATH_REDUCED, filename])
    image = cv2.imread(imagePath)

    (h, w) = image.shape[:2]

    # scale the bounding box coordinates relative to the spatial dimensions of the input image
    startX = xmin / w
    startY = ymin / h
    endX = xmax / w
    endY = ymax / h

    # load the image and preprocess it
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))

    # update our list of data, class labels, bounding boxes, and image paths
    data.append(image)
    labels.append(label)
    bboxes.append((startX, startY, endX, endY))
    imagePaths.append(imagePath)

For faster processing of data, the lists are converted into numpy arrays. Since the labels are in string format, we use scikit-learn’s LabelEncoder to transform them into their respective indices.

In [17]:
# convert the data, class labels, bounding boxes, and image paths to
# NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)
bboxes = np.array(bboxes, dtype="float32")
imagePaths = np.array(imagePaths)
# perform label encoding on the labels
le = LabelEncoder()
labels = le.fit_transform(labels)

In [23]:
# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

# Initialize lists to store predictions and ground truth
predictions = []
ground_truth = []

for image in imagePaths:
    # Load image
    img = cv2.imread(image)
    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(labels[class_ids[i]])
            cv2.rectangle(img, (x, y), (x + w, y + h), (0,255,0), 2)
            cv2.putText(img, label, (x, y + 30), cv2.FONT_HERSHEY_PLAIN, 3, (0,255,0), 2)
            
    '''plt.figure()
    plt.imshow(img)
    plt.title('Image')
    plt.axis('off')
    plt.show()'''

    # Store predicted labels
    predicted_labels = [labels[i] for i in class_ids]
    #TODO: falta sacar las reales y probar todo
    


In [25]:

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
(trainData, testData, trainLabels, testLabels) = train_test_split(
    data, labels, test_size=0.2, random_state=42
)

# Preprocess the data and labels
trainData /= 255.0
testData /= 255.0
trainLabels = to_categorical(trainLabels, num_classes=len(le.classes_))
testLabels = to_categorical(testLabels, num_classes=len(le.classes_))

# Create a simple CNN model (you may need to adjust the architecture based on your data)
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(le.classes_), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(trainData, trainLabels, epochs=10, batch_size=32, validation_split=0.1)

# Evaluate the model on the test set
predictions = model.predict(testData)
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy and other metrics
accuracy = accuracy_score(np.argmax(testLabels, axis=1), predicted_labels)
classification_report_str = classification_report(np.argmax(testLabels, axis=1), predicted_labels, target_names=le.classes_)
confusion_mat = confusion_matrix(np.argmax(testLabels, axis=1), predicted_labels)

# Print the metrics
print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_report_str)
print("Confusion Matrix:\n", confusion_mat)


Epoch 1/10
90/90 [==============================] - 78s 767ms/step - loss: 2.6541 - accuracy: 0.1788 - val_loss: 2.5196 - val_accuracy: 0.2375
Epoch 2/10
90/90 [==============================] - 63s 700ms/step - loss: 2.3378 - accuracy: 0.2531 - val_loss: 2.2720 - val_accuracy: 0.2656
Epoch 3/10
90/90 [==============================] - 64s 707ms/step - loss: 1.9919 - accuracy: 0.3684 - val_loss: 2.2181 - val_accuracy: 0.3156
Epoch 4/10
90/90 [==============================] - 64s 707ms/step - loss: 1.4277 - accuracy: 0.5396 - val_loss: 2.4942 - val_accuracy: 0.3187
Epoch 5/10
90/90 [==============================] - 64s 707ms/step - loss: 0.8084 - accuracy: 0.7424 - val_loss: 3.0815 - val_accuracy: 0.2594
Epoch 6/10
90/90 [==============================] - 64s 708ms/step - loss: 0.3854 - accuracy: 0.8764 - val_loss: 4.1009 - val_accuracy: 0.2844
Epoch 7/10
90/90 [==============================] - 64s 708ms/step - loss: 0.1849 - accuracy: 0.9476 - val_loss: 5.9183 - val_accuracy: 0.2688

C:\Users\Usuario\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Usuario\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Usuario\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r